In [8]:
import requests
from requests.exceptions import ConnectionError
from http.client import RemoteDisconnected
from bs4 import BeautifulSoup
import time
import html5lib
import json
import random

In [17]:
urls = [
    'https://next.rikunabi.com/kw%91%E5%8DH/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%83h%83%89%83C%83o%81%255B/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%92%CA%90M+%8DH%8E%96/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%83%255C%83t%83g%83E%83F%83A+%8AJ%94%AD/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%96%F2%8D%DC%8Et/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8E%F3%95t+%88%C4%93%E0/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8A%C5%8C%EC%8Et/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%95%DB%88%E7%8Em/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%8Cx%94%F5/?keywordsearch=1&log_f=1',
    'https://next.rikunabi.com/kw%94%FC%97e%8Et/?keywordsearch=1&log_f=1'
    ]
fileNames = [
    'carpentor',
    'driver',
    'electrician',
    'software_engineer',
    'pharmacist',
    'secretary',
    'nurse',
    'childminder',
    'security_guard',
    'beautician'
    ]

In [10]:
jobNumberFinder = 'rnn-pageNumber rnn-textXl'
jobTitleFinder = "rnn-linkText rnn-linkText--black"
jobDetailBaseUrl = "https://next.rikunabi.com"
jobDetailDescriptionFinder = 'rn3-companyOfferRecruitment__headText'
jobDescriptionAndWantedEmployeeFinder = '.rn3-companyOfferRecruitment__text'

In [18]:
fileNameindex = 0
failedTopUrls = []
failedDetailUrls = []
errors = []
for url in urls:
    htmlNumber = 1
    data = []
    dataIndex = 1
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html5lib")
        time.sleep(1)
        jobNumber = soup.find("span", class_=jobNumberFinder).get_text()
        print("Success - url:" + url + "jobNumber:" + jobNumber)
    except (RemoteDisconnected, ConnectionError, Exception) as e:
        print('Fail - url:' + url)
        failedTopUrls.append(url)
        errors.append(e)
        time.sleep(0.3)
        continue
    while(True):
        if(htmlNumber == 1):
            titles = soup.find_all("a", class_=jobTitleFinder)
        else:
            try:
                newUrl = url.replace('(?<=\?)(.*)(?=\/)', 'crn' + str(htmlNumber) + '.html')
                response = requests.get(newUrl)                
                time.sleep(1)
                soup = BeautifulSoup(response.content, "html5lib")
                response.close()
                titles = soup.find_all("a", class_=jobTitleFinder)
            except (RemoteDisconnected, ConnectionError, Exception) as e:
                print('Fail - url:' + newUrl)
                errors.append(e)
                failedTopUrls.append(newUrl)
                htmlNumber = htmlNumber + 50
                if(htmlNumber - 1 >= int(jobNumber)):
                    break
                time.sleep(random.randint(2,5))
                continue
        for title in titles:
            try:
                jobDetailResponse = requests.get(jobDetailBaseUrl + title['href'])                
                time.sleep(1)
                jobDetailSoup = BeautifulSoup(jobDetailResponse.content, "html5lib")
                jobDetailResponse.close()
                jobDescriptionSummary = None if jobDetailSoup.find("div", class_=jobDetailDescriptionFinder) is None else jobDetailSoup.find("div", class_=jobDetailDescriptionFinder).get_text()
                if jobDescriptionSummary is not None:
                    jobDescriptions = jobDetailSoup.select(jobDescriptionAndWantedEmployeeFinder)
                    jobDescriptionText = ''
                    jobWantedPersonText = ''
                    descriptionIndex = 0
                    for jobDescription in jobDescriptions:
                        if(descriptionIndex == 0):
                            jobDescriptionText = jobDescription.get_text()
                        elif(descriptionIndex == 1):
                            jobWantedPersonText = jobDescription.get_text()
                            break
                        descriptionIndex = descriptionIndex + 1                    
            except (RemoteDisconnected, ConnectionError, Exception) as e:
                print('url:' + jobDetailBaseUrl + title['href'])
                failedDetailUrls.append(newUrl)
                errors.append(e)
                continue
            if jobDescriptionSummary is None:
                try:
                    jobMoreDetailResponse = requests.get(jobDetailBaseUrl + title['href'].replace('nx1', 'nx2'))
                    time.sleep(1)
                    jobDetailMoreSoup = BeautifulSoup(jobMoreDetailResponse.content, "html5lib")
                    jobMoreDetailResponse.close()                
                    jobDescriptionSummary = jobDetailMoreSoup.find("div", class_=jobDetailDescriptionFinder).get_text()
                    if jobDescriptionSummary is not None:
                        jobDescriptions = jobDetailMoreSoup.select(jobDescriptionAndWantedEmployeeFinder)
                        jobDescriptionText = ''
                        jobWantedPersonText = ''
                        descriptionIndex = 0                        
                        for jobDescription in jobDescriptions:
                            if(descriptionIndex == 0):
                                jobDescriptionText = jobDescription.get_text()
                            elif(descriptionIndex == 1):
                                jobWantedPersonText = jobDescription.get_text()
                                break
                            descriptionIndex = descriptionIndex + 1                    
                except (RemoteDisconnected, ConnectionError, Exception) as e:
                    print('url:' + jobDetailBaseUrl + title['href'].replace('nx1', 'nx2'))
                    errors.append(e)
                    continue
                
            data.append(
                {str(dataIndex) : {
                    'title': title.get_text(),
                    'description_summary':jobDescriptionSummary,
                    'description_detail':jobDescriptionText,
                    'desired_person':jobWantedPersonText
                    }})
            dataIndex = dataIndex + 1
            time.sleep(random.randint(2,5))
        htmlNumber = htmlNumber + 50
        if(htmlNumber - 1 >= int(jobNumber)):
            break
        time.sleep(random.randint(3,5))
    
    # export data into json file
    with open('./data/job_advertisements/ja/' + fileNames[fileNameindex] + '.json', mode='w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=2)
    print(fileNames[fileNameindex] + ':done')
    fileNameindex = fileNameindex + 1
    
# export Top Url data into json file
with open('./data/job_advertisements/ja/failedTopUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedTopUrls, file, ensure_ascii=False, indent=2)
# export detailed Url data into json file
with open('./data/job_advertisements/ja/failedDetailUrls.json', mode='w', encoding='utf-8') as file:
    json.dump(failedDetailUrls, file, ensure_ascii=False, indent=2)

Success - url:https://next.rikunabi.com/kw%94%FC%97e%8Et/?keywordsearch=1&log_f=1jobNumber:182
beautician:done
